# Importing data and loading packages 
First, I am going to import libraries required and read the data into python.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
from pandas import ExcelWriter
from pandas import ExcelFile
from dateutil.parser import parse 
import re


### INDEED 

In [2]:
indeed= pd.read_csv("file:///Users/manha/Desktop/DSE_6000/Active_Class/Project/indeed/indeed_data_scientist.csv")
indeed.drop('Date', axis=1, inplace=True)

#zip =pd.read_csv("file:///Users/manha/Desktop/DSE_6000/Active_Class/Project/Zip/zip_recruiter_company.csv")
#Combining Zip and indeed datafile
#combined = pd.concat([zip], axis=0,sort=False)
#indeed = pd.concat([indeed, zip], axis=0, sort=False)
#indeed.tail()

In [3]:
print(indeed.shape)
indeed.info()
#indeed.head()


(9349, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9349 entries, 0 to 9348
Data columns (total 5 columns):
Job_Title           9349 non-null object
Company_Name        9349 non-null object
Location            4409 non-null object
Job_Description     9349 non-null object
Estimated_Salary    1181 non-null object
dtypes: object(5)
memory usage: 365.3+ KB


### Eliminating Duplicate Job Postings
I began by reading in my scraped data from my saved csv, and checking its shape. I then eliminated 
all duplicate rows, and assessed how many truly unique postings I gathered in my scraping.

In [4]:
#dropping any duplicate rows:
indeed = indeed.drop_duplicates()
indeed.reset_index(drop=True, inplace=True)
indeed.shape
#indeed.head()

(1817, 5)

From this process alone, my data frame was reduced from 1348 rows down to 123. While not 
complex in terms of coding, I only wanted to use unique job postings when conducting my future 
analysis.

## Formatting Salary Data
To prepare the salary data, I first wanted to look at the sorts of values I was dealing with:

In [5]:
#indeed['Estimated_Salary'].value_counts()


***As you can see, there are several aspects of this data that could make further calculations difficult.
Python doesn’t know how to handle characters like commas and dollar-signs when conducting mathematical 
operations, so we need to get rid of these and “\n” strings before we can go much further.
On top of this, you’ll notice that salaries can be presented as single figures, or ranges, and that
they can appear on different scales of payment — per year, per month, or per hour. I needed to take 
care of all of these items in the course of preparing my salary data for further analysis.
I began by stripping out some of the extraneous characters that I knew I needed to eliminate: ***

In [6]:
#some targeted cleaning of salary information to make parsing easier #—> remove “\n”, “$”, and “,”
indeed['Estimated_Salary'] = indeed['Estimated_Salary'].str.replace('\n', '')
indeed['Estimated_Salary'] = indeed['Estimated_Salary'].str.replace(',', '')
indeed['Estimated_Salary']= indeed['Estimated_Salary'].str.replace('$', '')
indeed['Estimated_Salary']= indeed['Estimated_Salary'].str.replace('From', '')
#indeed.tail()

While I knew I would need to remove string data on the scale of pay (e.g., “a year,” “a month”) from the salary data before I could work with it, I did want to preserve information about these original pay scales for possible future use. As such, I created a new column of data to capture this. I called this new column “og_salary_period”:

In [7]:
indeed['salary_period'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
indeed.loc[indeed['Estimated_Salary'].str.contains('year',na=False), 'salary_period'] ='year'
indeed.loc[indeed['Estimated_Salary'].str.contains('month',na=False), 'salary_period'] ='month'
indeed.loc[indeed['Estimated_Salary'].str.contains('week',na=False), 'salary_period'] = 'week'
indeed.loc[indeed['Estimated_Salary'].str.contains('day',na=False), 'salary_period'] = 'day'
indeed.loc[indeed['Estimated_Salary'].str.contains('hour', na=False), 'salary_period'] ='hour'
indeed.loc[indeed['salary_period'].str.contains('NaN',na=True), 'salary_period'] ='Nothing_found'

indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period
0,"Director II, Data Science, GRM Product Design ...",Liberty Mutual Insurance,"Seattle, WA 98101 (Downtown area)",Maintain a knowledge of the latest development...,150200 - 197600 a year,year
1,Data Engineer,Seen by Indeed,"St. Louis, MO",Get started – Complete a 5-minute application ...,NaN,Nothing_found
2,Business Analyst - Data Science,MedPro Group,"Fort Wayne, IN 46815 (Maplewood Park area)",Participate in developing functional requireme...,NaN,Nothing_found
3,"Senior Analyst, Data Science Integrations",Wayfair,"Boston, MA","Senior Analyst, Data Science Integrations.The ...",NaN,Nothing_found
4,Data Scientist,Brains Workgroup,"Cambridge, MA 02140",2+ years of industry experience in a Data Scie...,NaN,Nothing_found


In [8]:
indeed.loc[indeed['Estimated_Salary'].str.contains('NaN',na=True), 'Estimated_Salary'] ='Nothing_found'
indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period
0,"Director II, Data Science, GRM Product Design ...",Liberty Mutual Insurance,"Seattle, WA 98101 (Downtown area)",Maintain a knowledge of the latest development...,150200 - 197600 a year,year
1,Data Engineer,Seen by Indeed,"St. Louis, MO",Get started – Complete a 5-minute application ...,Nothing_found,Nothing_found
2,Business Analyst - Data Science,MedPro Group,"Fort Wayne, IN 46815 (Maplewood Park area)",Participate in developing functional requireme...,Nothing_found,Nothing_found
3,"Senior Analyst, Data Science Integrations",Wayfair,"Boston, MA","Senior Analyst, Data Science Integrations.The ...",Nothing_found,Nothing_found
4,Data Scientist,Brains Workgroup,"Cambridge, MA 02140",2+ years of industry experience in a Data Scie...,Nothing_found,Nothing_found


In [9]:
#indeed['Estimated_Salary'].value_counts()

In [10]:
#salary_data = indeed[indeed['Estimated_Salary'] != 'Nothing_found']
#indeed.head()
#print(salary_data )

I then isolated all jobs with salary information into a separate data frame that I could work with to scale them all accordingly. Notice, I also removed these salaried rows entirely from the original scale_data table. I did this knowing that I would re-append the salaried rows once I had effectively standardized their appearance.

In [11]:
#filtering out the results with salary data != 'Nothing'
salary_data = indeed[indeed['Estimated_Salary'] != 'Nothing_found']
#removing all rows in salary data from scrape data, and converting
#all 'Nothing_found' values to NaN, so that float salary values can
#be easily reintegrated later
#indeed = [indeed['Estimated_Salary'][~indeed.isin(salary_data)].dropna(how ='all')indeed['Estimated_Salary'].replace("Nothing", np.nan, inplace=True)
#indeed['Estimated_Salary'].astype('float')
indeed =indeed[~indeed.isin(salary_data)].dropna(how='all')
          
indeed['Estimated_Salary'].replace('Nothing_found',np.nan, inplace=True)
indeed['Estimated_Salary'].astype('float')  
indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period
1,Data Engineer,Seen by Indeed,"St. Louis, MO",Get started – Complete a 5-minute application ...,NaN,Nothing_found
2,Business Analyst - Data Science,MedPro Group,"Fort Wayne, IN 46815 (Maplewood Park area)",Participate in developing functional requireme...,NaN,Nothing_found
3,"Senior Analyst, Data Science Integrations",Wayfair,"Boston, MA","Senior Analyst, Data Science Integrations.The ...",NaN,Nothing_found
4,Data Scientist,Brains Workgroup,"Cambridge, MA 02140",2+ years of industry experience in a Data Scie...,NaN,Nothing_found
5,Marketing Data Science Analyst (SQL/Python/Tab...,Tractor Supply Company,"Brentwood, TN 37027",The Marketing Data Science Analyst is responsi...,NaN,Nothing_found


Here’s a quick look at the salary data frame. You’ll notice that the og_salary_period column captures information about the original scale of the salary information.

In [12]:

year_salaries = salary_data[salary_data["Estimated_Salary"].str.contains('year',na=False)]
year_salaries["Estimated_Salary"] = year_salaries["Estimated_Salary"].str.replace('a year', '')
hour_salaries = salary_data[salary_data["Estimated_Salary"].str.contains('hour',na=False)]
#year_salaries.head()


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


From here, I separated job postings with salary information according to the scale in which original salary data was presented. I also removed the string data associated with the pay scale.

In [13]:
#separating out data based on whether contains hour, day, week, #month, year
year_salaries = salary_data[salary_data["Estimated_Salary"].str.contains('year',na=False)]
month_salaries = salary_data[salary_data["Estimated_Salary"].str.contains('month',na=False)]
week_salaries = salary_data[salary_data["Estimated_Salary"].str.contains('week',na=False)]
day_salaries = salary_data[salary_data["Estimated_Salary"].str.contains('day',na=False)]
hour_salaries = salary_data[salary_data["Estimated_Salary"].str.contains('hour',na=False)]


In [14]:
# removing string values(“ a year”, “ a week”, etc. from salary dfs)
year_salaries["Estimated_Salary"] = year_salaries["Estimated_Salary"].str.replace('a year', '')
month_salaries["Estimated_Salary"] = month_salaries["Estimated_Salary"].str.replace(' a month', '')
week_salaries["Estimated_Salary"] = week_salaries["Estimated_Salary"].str.replace('a week', '')
day_salaries["Estimated_Salary"] = day_salaries["Estimated_Salary"].str.replace('a day', '')
hour_salaries["Estimated_Salary"] = hour_salaries["Estimated_Salary"].str.replace('an hour', '')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [15]:
salary_data.head()
salary_data.shape


(229, 6)

In [16]:
def split_sal(i):
    #print(i)
    #type(i)
    try:
        splt = i.split('-',1)
        #print(splt)
        first = float(splt[0])
        #print(first)
        second = float(splt[1])
        #print(second)
        return (first + second)/2
    except:
        return float(i)


    
year_salaries['Estimated_Salary'] = year_salaries['Estimated_Salary'].apply(split_sal)
month_salaries['Estimated_Salary'] = month_salaries['Estimated_Salary'].apply(split_sal)
month_salaries['Estimated_Salary'] = month_salaries['Estimated_Salary'] * 12
week_salaries['Estimated_Salary'] = week_salaries['Estimated_Salary'].apply(split_sal)
week_salaries['Estimated_Salary'] = week_salaries['Estimated_Salary'] * 52
day_salaries['Estimated_Salary'] = day_salaries['Estimated_Salary'].apply(split_sal)
day_salaries['Estimated_Salary'] = day_salaries['Estimated_Salary'] * 260
hour_salaries['Estimated_Salary'] = hour_salaries['Estimated_Salary'].apply(split_sal)
hour_salaries['Estimated_Salary'] = hour_salaries['Estimated_Salary'] * 2080
#year_salaries.head()
#indeed.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

With my function in place, I could then clean the salary data and transform any salary data that wasn’t on an annual scale to approximate yearly earnings.
(While there may be discussion to be had regarding whether to keep jobs with hourly and weekly salaries, I decided to preserve these jobs for the sake of having more data available, and because I felt that the pay grade could still approximate the true value of the work on offer.)

In [17]:
#rejoining salary data into main scrape_data df
combined_salaries = pd.concat([year_salaries,month_salaries, week_salaries, day_salaries, hour_salaries], axis=0)
indeed = pd.concat([indeed, combined_salaries], axis=0)
indeed.tail()


,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period
1571,Senior Data Engineer,digital dhara,NaN,Work with Data Scientists and build scripts to...,135200.0,hour
1602,Data Systems Analyst,GLIDE MEMORIAL CHURCH,NaN,"Experience with statistical, graphical, and da...",52000.0,hour
1630,Data Scientist,Employment Placement Agency,NaN,Learn from peers in data science and engineeri...,135200.0,hour
1731,Data Scientists – Machine Learning,Brielle Technologies,NaN,You will work on Advanced Analytics using Big ...,135200.0,hour
1769,Part Time Data Scientist (Geospatial),BCC Geospatial Center of the CUNY CREST Institute,NaN,S/He must be passionate and enthusiastic about...,46800.0,hour


In [18]:
### Further data cleaning

In [19]:
#creating data character cleaning function, and applying to all 
#columns, also lowercasing all string data for ease of later nlp
def data_clean(df, column):
    cleaning_list = ['+', '$','/',',','?','.',';','-','@','!','&','%','^','*',')','(', '\n']
    for item in cleaning_list:
        df[column] = df[column].str.replace(item, '')
##can’t clean the salary column due to float values, and don’t need 
#to clean og_salary, so keeping out of the for loop
for column in indeed.columns[0:len(indeed.columns)-2]:
    data_clean(indeed, column)
#indeed.head()

In [20]:
#indeed['Location'].value_counts()

In [21]:
#creating data character cleaning function, and applying to all 

cleaning_list = ['15237', '32801','Central Business District area','North area','77022','94612','80538','94612','46815','02140','01760','98101','48326'
                ,'Downtown area','48326','57104','80301','37027','80538','02101','94105',' Financial District area',
                '85705 Flowing Wells area','80021','02451','55423','Maplewood Park area']
for item in cleaning_list:
    indeed["Location"] = indeed["Location"].str.replace(item,'' )

#indeed.head()
indeed['Location'].value_counts()

St Louis MO           17
Oakland CA            10
Boston MA             10
Cambridge MA           9
Natick MA              9
Fort Wayne IN          9
Sioux Falls SD         7
Auburn Hills MI        7
Seattle WA             7
Boulder CO             5
Kansas City MO         4
Brentwood TN           3
Chevy Chase MD         3
Seattle WA             3
Tucson AZ              2
San Francisco CA       2
Boston MA              2
Auburn Hills MI        2
Westminster CO         2
Loveland CO            2
New York NY            2
Monmouth County NJ     2
Richfield MN           1
Washington DC          1
Denver CO              1
Scott AFB IL           1
Austin TX              1
Redlands CA            1
Waltham MA             1
Name: Location, dtype: int64

In [22]:
indeed['State'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
indeed.loc[indeed['Location'].str.contains('NY',na=False), 'State'] ='NY'
indeed.loc[indeed['Location'].str.contains('PA',na=False), 'State'] ='PA'
indeed.loc[indeed['Location'].str.contains('MA',na=False), 'State'] ='MA'
indeed.loc[indeed['Location'].str.contains('TX',na=False), 'State'] ='TX'
indeed.loc[indeed['Location'].str.contains('FL',na=False), 'State'] ='FL'
indeed.loc[indeed['Location'].str.contains('MO',na=False), 'State'] ='MO'
indeed.loc[indeed['Location'].str.contains('IN',na=False), 'State'] ='IN'
indeed.loc[indeed['Location'].str.contains('MN',na=False), 'State'] ='MN'
indeed.loc[indeed['Location'].str.contains('CO',na=False), 'State'] ='CO'
indeed.loc[indeed['Location'].str.contains('FL',na=False), 'State'] ='DC'
indeed.loc[indeed['Location'].str.contains('NJ',na=False), 'State'] ='NJ'
indeed.loc[indeed['Location'].str.contains('AZ',na=False), 'State'] ='AZ'
indeed.loc[indeed['Location'].str.contains('MI',na=False), 'State'] ='MI'
indeed.loc[indeed['Location'].str.contains('MD',na=False), 'State'] ='MD'
indeed.loc[indeed['Location'].str.contains('WA',na=False), 'State'] ='WA'
indeed.loc[indeed['Location'].str.contains('TN',na=False), 'State'] ='TN'
indeed.loc[indeed['Location'].str.contains('SD',na=False), 'State'] ='SD'
indeed.loc[indeed['Location'].str.contains('WA',na=False), 'State'] ='WA'
indeed.loc[indeed['Location'].str.contains('CA',na=False), 'State'] ='CA'
indeed.loc[indeed['State'].str.contains('NaN',na=True), 'State'] ='Not Mention'
indeed.head()
#indeed['Location'].value_counts()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State
1,Data Engineer,Seen by Indeed,St Louis MO,Get started – Complete a 5minute application t...,NaN,Nothing_found,MO
2,Business Analyst Data Science,MedPro Group,Fort Wayne IN,Participate in developing functional requireme...,NaN,Nothing_found,IN
3,Senior Analyst Data Science Integrations,Wayfair,Boston MA,Senior Analyst Data Science IntegrationsThe te...,NaN,Nothing_found,MA
4,Data Scientist,Brains Workgroup,Cambridge MA,2 years of industry experience in a Data Scien...,NaN,Nothing_found,MA
5,Marketing Data Science Analyst SQLPythonTablea...,Tractor Supply Company,Brentwood TN,The Marketing Data Science Analyst is responsi...,NaN,Nothing_found,TN


In [23]:
indeed['City'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
indeed.loc[indeed['Location'].str.contains('New York',na=False), 'City'] ='New York'
indeed.loc[indeed['Location'].str.contains('Pittsburgh',na=False), 'City'] ='Pittsburgh'
indeed.loc[indeed['Location'].str.contains('Orlando',na=False), 'City'] ='Orlando'
indeed.loc[indeed['Location'].str.contains('Philadelphia',na=False), 'City'] ='Philadelphia'
indeed.loc[indeed['Location'].str.contains('Boston',na=False), 'City'] ='Boston'
indeed.loc[indeed['Location'].str.contains('Houston',na=False), 'City'] ='Houston'
indeed.loc[indeed['Location'].str.contains('St Louis',na=False), 'City'] ='St Louis'
indeed.loc[indeed['Location'].str.contains('Oakland',na=False), 'City'] ='Oakland'
indeed.loc[indeed['Location'].str.contains('Boston',na=False), 'City'] ='Boston'
indeed.loc[indeed['Location'].str.contains('Fort Wayne',na=False), 'City'] ='Fort Wayne'
indeed.loc[indeed['Location'].str.contains('Cambridge',na=False), 'City'] ='Cambridge'
indeed.loc[indeed['Location'].str.contains('Natick',na=False), 'City'] ='Natick'
indeed.loc[indeed['Location'].str.contains('Seattle',na=False), 'City'] ='Seattle'
indeed.loc[indeed['Location'].str.contains('Sioux Falls',na=False), 'City'] ='Sioux Falls'
indeed.loc[indeed['Location'].str.contains('Auburn Hills',na=False), 'City'] ='Auburn Hills'
indeed.loc[indeed['Location'].str.contains('Boulder',na=False), 'City'] ='Boulder'
indeed.loc[indeed['Location'].str.contains('Kansas City',na=False), 'City'] ='Kansas City'
indeed.loc[indeed['Location'].str.contains('Brentwood',na=False), 'City'] ='Brentwood'
indeed.loc[indeed['Location'].str.contains('Chevy Chase',na=False), 'City'] ='Chevy Chase'
indeed.loc[indeed['Location'].str.contains('Westminster',na=False), 'City'] ='Westminster'
indeed.loc[indeed['Location'].str.contains('Tucson',na=False), 'City'] ='Tucson'
indeed.loc[indeed['Location'].str.contains('San Francisco',na=False), 'City'] ='San Francisco'
indeed.loc[indeed['Location'].str.contains('Monmouth County',na=False), 'City'] ='Monmouth County'
indeed.loc[indeed['Location'].str.contains('Loveland',na=False), 'City'] ='Loveland'
indeed.loc[indeed['Location'].str.contains('Redlands',na=False), 'City'] ='Redlands'
indeed.loc[indeed['Location'].str.contains('Washington',na=False), 'City'] ='Washington'
indeed.loc[indeed['Location'].str.contains('Austin',na=False), 'City'] ='Austin'
indeed.loc[indeed['Location'].str.contains('Denver',na=False), 'City'] ='Denver'
indeed.loc[indeed['Location'].str.contains('Waltham',na=False), 'City'] ='Waltham'
indeed.loc[indeed['Location'].str.contains('Richfield',na=False), 'City'] ='Richfield'
indeed.loc[indeed['Location'].str.contains('Scott AFB',na=False), 'City'] ='Scott AFB'
indeed.loc[indeed['City'].str.contains('NaN',na=True), 'City'] ='Not Mention'

indeed.head()
#indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City
1,Data Engineer,Seen by Indeed,St Louis MO,Get started – Complete a 5minute application t...,NaN,Nothing_found,MO,St Louis
2,Business Analyst Data Science,MedPro Group,Fort Wayne IN,Participate in developing functional requireme...,NaN,Nothing_found,IN,Fort Wayne
3,Senior Analyst Data Science Integrations,Wayfair,Boston MA,Senior Analyst Data Science IntegrationsThe te...,NaN,Nothing_found,MA,Boston
4,Data Scientist,Brains Workgroup,Cambridge MA,2 years of industry experience in a Data Scien...,NaN,Nothing_found,MA,Cambridge
5,Marketing Data Science Analyst SQLPythonTablea...,Tractor Supply Company,Brentwood TN,The Marketing Data Science Analyst is responsi...,NaN,Nothing_found,TN,Brentwood


In [24]:
indeed['Python'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
indeed.loc[indeed['Job_Description'].str.contains('Python',na=False), 'Python'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('python',na=False), 'Python'] ='1'
indeed.loc[indeed['Python'].str.contains('NaN',na=True), 'Python'] ='o'
indeed['SQL'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('SQL',na=False), 'SQL'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('sql',na=False), 'SQL'] ='1'
indeed.loc[indeed['SQL'].str.contains('NaN',na=True), 'SQL'] ='0'
indeed['NoSQL'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('NoSQL',na=False), 'NoSQL'] ='1'
indeed.loc[indeed['NoSQL'].str.contains('NaN',na=True), 'NoSQL'] ='0'
indeed['java'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('java',na=False), 'java'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('JavaScript',na=False), 'java'] ='1'
indeed.loc[indeed['java'].str.contains('NaN',na=True), 'java'] ='o'
indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City,Python,SQL,NoSQL,java
1,Data Engineer,Seen by Indeed,St Louis MO,Get started – Complete a 5minute application t...,NaN,Nothing_found,MO,St Louis,o,0,0,o
2,Business Analyst Data Science,MedPro Group,Fort Wayne IN,Participate in developing functional requireme...,NaN,Nothing_found,IN,Fort Wayne,o,0,0,o
3,Senior Analyst Data Science Integrations,Wayfair,Boston MA,Senior Analyst Data Science IntegrationsThe te...,NaN,Nothing_found,MA,Boston,o,0,0,o
4,Data Scientist,Brains Workgroup,Cambridge MA,2 years of industry experience in a Data Scien...,NaN,Nothing_found,MA,Cambridge,o,0,0,o
5,Marketing Data Science Analyst SQLPythonTablea...,Tractor Supply Company,Brentwood TN,The Marketing Data Science Analyst is responsi...,NaN,Nothing_found,TN,Brentwood,o,0,0,o


In [25]:
indeed['swift'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Swift',na=False), 'swift'] ='1'
indeed.loc[indeed['swift'].str.contains('NaN',na=True), 'swift'] ='0'
indeed['PHP'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('PHP',na=False), 'PHP'] ='1'
indeed.loc[indeed['PHP'].str.contains('NaN',na=True), 'PHP'] ='0'

indeed['R'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('R',na=False), 'R'] ='1'
indeed.loc[indeed['R'].str.contains('NaN',na=True), 'R'] ='0'
indeed.head()

#indeed.head(100)

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City,Python,SQL,NoSQL,java,swift,PHP,R
1,Data Engineer,Seen by Indeed,St Louis MO,Get started – Complete a 5minute application t...,NaN,Nothing_found,MO,St Louis,o,0,0,o,0,0,0
2,Business Analyst Data Science,MedPro Group,Fort Wayne IN,Participate in developing functional requireme...,NaN,Nothing_found,IN,Fort Wayne,o,0,0,o,0,0,0
3,Senior Analyst Data Science Integrations,Wayfair,Boston MA,Senior Analyst Data Science IntegrationsThe te...,NaN,Nothing_found,MA,Boston,o,0,0,o,0,0,0
4,Data Scientist,Brains Workgroup,Cambridge MA,2 years of industry experience in a Data Scien...,NaN,Nothing_found,MA,Cambridge,o,0,0,o,0,0,0
5,Marketing Data Science Analyst SQLPythonTablea...,Tractor Supply Company,Brentwood TN,The Marketing Data Science Analyst is responsi...,NaN,Nothing_found,TN,Brentwood,o,0,0,o,0,0,0


In [26]:
indeed['Project_Management'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Project Management',na=False), 'Project_Management'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('ProjectManagement',na=False), 'Project_Management'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('Management',na=False), 'Project_Management'] ='1'
indeed.loc[indeed['Project_Management'].str.contains('NaN',na=True), 'Project_Management'] ='0'
indeed['Risk_Management'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Risk Management',na=False), 'Risk_Management'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('RiskManagement',na=False), 'Risk_Management'] ='1'
indeed.loc[indeed['Risk_Management'].str.contains('Nan',na=True), 'Risk_Management'] ='0'
indeed['Analysis_Skill'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Analysis Skill',na=False), 'Analysis_Skill'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('AnalysisSkill',na=False), 'Analysis_Skill'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('Analysis',na=False), 'Analysis_Skill'] ='1'
indeed.loc[indeed['Analysis_Skill'].str.contains('NaN',na=True), 'Analysis_Skill'] ='0'
indeed['Hadoop'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Hadoop',na=False), 'Hadoop'] ='1'
indeed.loc[indeed['Hadoop'].str.contains('NaN',na=True), 'Hadoop'] ='0'
indeed['SAS'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('SAS',na=False), 'SAS'] ='1'
indeed.loc[indeed['SAS'].str.contains('NaN',na=True), 'SAS'] ='0'
indeed['Microsoft_Word'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Microsoft Word',na=False), 'Microsoft_Word'] ='1'
indeed.loc[indeed['Microsoft_Word'].str.contains('NaN',na=True), 'Microsoft_Word'] ='0'
indeed['Microsoft_Office'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Microsoft Office',na=False), 'Microsoft_Office'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('Microsoft office',na=False), 'Microsoft_Office'] ='1'                                                 
indeed.loc[indeed['Microsoft_Office'].str.contains('NaN',na=True), 'Microsoft_Office'] = '0'
           

indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City,Python,SQL,...,swift,PHP,R,Project_Management,Risk_Management,Analysis_Skill,Hadoop,SAS,Microsoft_Word,Microsoft_Office
1,Data Engineer,Seen by Indeed,St Louis MO,Get started – Complete a 5minute application t...,NaN,Nothing_found,MO,St Louis,o,0,...,0,0,0,0,0,0,0,0,0,0
2,Business Analyst Data Science,MedPro Group,Fort Wayne IN,Participate in developing functional requireme...,NaN,Nothing_found,IN,Fort Wayne,o,0,...,0,0,0,0,0,0,0,0,0,0
3,Senior Analyst Data Science Integrations,Wayfair,Boston MA,Senior Analyst Data Science IntegrationsThe te...,NaN,Nothing_found,MA,Boston,o,0,...,0,0,0,0,0,0,0,0,0,0
4,Data Scientist,Brains Workgroup,Cambridge MA,2 years of industry experience in a Data Scien...,NaN,Nothing_found,MA,Cambridge,o,0,...,0,0,0,0,0,0,0,0,0,0
5,Marketing Data Science Analyst SQLPythonTablea...,Tractor Supply Company,Brentwood TN,The Marketing Data Science Analyst is responsi...,NaN,Nothing_found,TN,Brentwood,o,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
indeed['Hive'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Hive',na=False), 'Hive'] ='1'
indeed.loc[indeed['Hive'].str.contains('NaN',na=True), 'Hive'] ='0'
indeed['AWS'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('AWS',na=False), 'AWS'] ='1'
indeed.loc[indeed['AWS'].str.contains('NaN',na=True), 'AWS'] ='0'
indeed['Spark'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Spark',na=False), 'Spark'] ='1'
indeed.loc[indeed['Spark'].str.contains('NaN',na=True), 'Spark'] ='0'
indeed['Data_Mining'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Data Mining',na=False), 'Data_Mining'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('DataMining',na=False), 'Data_Mining'] ='1'
indeed.loc[indeed['Data_Mining'].str.contains('NaN',na=True), 'Data_Mining'] ='0'
indeed['Predictive_Analytics'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Predictive Analytics',na=False), 'Predictive_Analytics'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('PredictiveAnalytics',na=False), 'Predictive_Analytics'] ='1'
indeed.loc[indeed['Predictive_Analytics'].str.contains('NaN',na=True), 'Predictive_Analytics'] ='0'
indeed['Machine_Learning'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Machine Learning',na=False), 'Machine_Learning'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('MachineLearning',na=False), 'Machine_Learning'] ='1'

indeed.loc[indeed['Machine_Learning'].str.contains('NaN',na=True), 'Machine_Learning'] ='0'

indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City,Python,SQL,...,Hadoop,SAS,Microsoft_Word,Microsoft_Office,Hive,AWS,Spark,Data_Mining,Predictive_Analytics,Machine_Learning
1,Data Engineer,Seen by Indeed,St Louis MO,Get started – Complete a 5minute application t...,NaN,Nothing_found,MO,St Louis,o,0,...,0,0,0,0,0,0,0,0,0,0
2,Business Analyst Data Science,MedPro Group,Fort Wayne IN,Participate in developing functional requireme...,NaN,Nothing_found,IN,Fort Wayne,o,0,...,0,0,0,0,0,0,0,0,0,0
3,Senior Analyst Data Science Integrations,Wayfair,Boston MA,Senior Analyst Data Science IntegrationsThe te...,NaN,Nothing_found,MA,Boston,o,0,...,0,0,0,0,0,0,0,0,0,0
4,Data Scientist,Brains Workgroup,Cambridge MA,2 years of industry experience in a Data Scien...,NaN,Nothing_found,MA,Cambridge,o,0,...,0,0,0,0,0,0,0,0,0,0
5,Marketing Data Science Analyst SQLPythonTablea...,Tractor Supply Company,Brentwood TN,The Marketing Data Science Analyst is responsi...,NaN,Nothing_found,TN,Brentwood,o,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
indeed['Pig'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Pig',na=False), 'Pig'] ='1'
indeed.loc[indeed['Pig'].str.contains('NaN',na=True), 'Pig'] ='0'
indeed['Data_Analysis_Skill'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Data Analysis Skill',na=False), 'Data_Analysis_Skill'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('DataAnalysisSkill',na=False), 'Data_Analysis_Skill'] ='0'
indeed.loc[indeed['Job_Description'].str.contains('Data Analysis',na=False), 'Data_Analysis_Skill'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('DataAnalysis',na=False), 'Data_Analysis_Skill'] ='1'
indeed.loc[indeed['Data_Analysis_Skill'].str.contains('NaN',na=True), 'Data_Analysis_Skill'] ='0'
indeed['Communication_Skill'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Communication Skill',na=False), 'Communication_Skill'] ='1'
indeed.loc[indeed['Communication_Skill'].str.contains('NaN',na=True), 'Communication_Skill'] ='0'

indeed['Matlab'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Matlab',na=False), 'Matlab'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('matlab',na=False), 'Matlab'] ='1'
indeed.loc[indeed['Matlab'].str.contains('NaN',na=True), 'Matlab'] ='0'


In [29]:
indeed['Scala'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Scala',na=False), 'Scala'] ='1'
indeed.loc[indeed['Scala'].str.contains('NaN',na=True), 'Scala'] ='0'


In [30]:
#Education
indeed['Bachelor_Degree'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Bachelor',na=False), 'Bachelor_Degree'] ='1'
indeed.loc[indeed['Bachelor_Degree'].str.contains('NaN',na=True), 'Bachelor_Degree'] ='0'
indeed['Master_Degree'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Master Degree',na=False), 'Master_Degree'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('Master',na=False), 'Master_Degree'] ='1'
indeed.loc[indeed['Master_Degree'].str.contains('NaN',na=True), 'Master_Degree'] ='0'

indeed['Diploma_GED'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Diploma',na=False), 'Diploma_GED'] ='1'
indeed.loc[indeed['Job_Description'].str.contains('GED',na=False), 'Diploma_GED'] ='1'

indeed.loc[indeed['Diploma_GED'].str.contains('NaN',na=True), 'Diploma_GED'] ='0'

indeed['Doctoral_Degree'] = np.nan
indeed.loc[indeed['Job_Description'].str.contains('Doctoral',na=False), 'Doctoral_Degree'] ='1'
indeed.loc[indeed['Doctoral_Degree'].str.contains('NaN',na=True), 'Doctoral_Degree'] ='0'


In [31]:
indeed.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City,Python,SQL,...,Machine_Learning,Pig,Data_Analysis_Skill,Communication_Skill,Matlab,Scala,Bachelor_Degree,Master_Degree,Diploma_GED,Doctoral_Degree
1,Data Engineer,Seen by Indeed,St Louis MO,Get started – Complete a 5minute application t...,NaN,Nothing_found,MO,St Louis,o,0,...,0,0,0,0,0,0,0,0,0,0
2,Business Analyst Data Science,MedPro Group,Fort Wayne IN,Participate in developing functional requireme...,NaN,Nothing_found,IN,Fort Wayne,o,0,...,0,0,0,0,0,0,0,0,0,0
3,Senior Analyst Data Science Integrations,Wayfair,Boston MA,Senior Analyst Data Science IntegrationsThe te...,NaN,Nothing_found,MA,Boston,o,0,...,0,0,0,0,0,0,0,0,0,0
4,Data Scientist,Brains Workgroup,Cambridge MA,2 years of industry experience in a Data Scien...,NaN,Nothing_found,MA,Cambridge,o,0,...,0,0,0,0,0,0,0,0,0,0
5,Marketing Data Science Analyst SQLPythonTablea...,Tractor Supply Company,Brentwood TN,The Marketing Data Science Analyst is responsi...,NaN,Nothing_found,TN,Brentwood,o,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
indeed.to_csv('indeed.csv')

In [33]:
#indeed.info()
print(indeed.shape)

(1817, 37)


In [34]:
indeed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1817 entries, 1 to 1769
Data columns (total 37 columns):
Job_Title               1817 non-null object
Company_Name            1817 non-null object
Location                126 non-null object
Job_Description         1817 non-null object
Estimated_Salary        229 non-null float64
salary_period           1817 non-null object
State                   1817 non-null object
City                    1817 non-null object
Python                  1817 non-null object
SQL                     1817 non-null object
NoSQL                   1817 non-null object
java                    1817 non-null object
swift                   1817 non-null object
PHP                     1817 non-null object
R                       1817 non-null object
Project_Management      1817 non-null object
Risk_Management         1817 non-null object
Analysis_Skill          1817 non-null object
Hadoop                  1817 non-null object
SAS                     1817 non-null o